In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv("../input/vehicle-dataset-from-cardekho/CAR DETAILS FROM CAR DEKHO.csv")
data

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

In [ ]:
data.fuel.value_counts()

In [ ]:
data.seller_type.value_counts()

In [ ]:
data.transmission.value_counts()

In [ ]:
data.owner.value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data.fuel=le.fit_transform(data.fuel)
data.seller_type=le.fit_transform(data.seller_type)
data.transmission=le.fit_transform(data.transmission)
data.owner=le.fit_transform(data.owner)
data

In [ ]:
data['No of Years']=2021-data.year
data

In [ ]:
data=data.drop(['name','year'],axis=1)
data

In [ ]:
corr=data.corr()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(12,6))
sns.heatmap(corr,annot=True)

In [ ]:
sns.pairplot(data)

In [ ]:
X=data.drop(['selling_price','owner'],axis=1)
y=data.selling_price
X.shape,y.shape

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
et = ExtraTreesRegressor()
et.fit(X,y)

In [ ]:
et.feature_importances_

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(x=X.columns,y=et.feature_importances_)
plt.show

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)
X_train.shape,X_test.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train,y_train)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rsc = RandomizedSearchCV(RandomForestRegressor(),{
    'n_estimators':[int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)],
    'max_features':['auto','sqrt'],
    'max_depth':[int(x) for x in np.linspace(5,30, num = 6)],
    'min_samples_split':[2,5,10,15,100],
    'min_samples_leaf':[1,2,5,10]}
)

In [ ]:
rsc.fit(X_train,y_train)

In [ ]:
rsc.best_params_

In [ ]:
param = pd.DataFrame(rsc.cv_results_)
param

In [ ]:
param[['param_n_estimators','param_min_samples_split',
       'param_min_samples_leaf','param_max_features','param_max_depth','mean_test_score']]

In [ ]:
rf = RandomForestRegressor(n_estimators= 1100,
 min_samples_split=5,
 min_samples_leaf= 2,
 max_features='sqrt',
 max_depth= 30)
rf.fit(X_train,y_train)

In [ ]:
from sklearn import metrics
pred=rf.predict(X_test)
score = metrics.r2_score(y_test,pred)
score

In [ ]:
plt.scatter(y_test,pred)